In [1]:
import pandas as pd

import numpy as np

In [2]:
raw_df = pd.read_csv("~/Desktop/thesis/data/Adams_experiment_cleaned_filtered.csv")
raw_df["last"] = ~raw_df["did_continue_eval"]
n = 10000 #len(raw_df["subject"].unique())
months = raw_df.groupby("word").agg({"s2_value": "mean"}).reset_index()

In [3]:
# Output: last, subject.id, value, order

df = pd.DataFrame({"subject.id": range(n)})
df = df.merge(months, how="cross").groupby("subject.id").sample(frac=1).reset_index(drop=True)
df["order"] = df.groupby("subject.id").cumcount()+1
df = df.rename(columns={"s2_value": "value"})
df.head(16)

,subject.id,word,value,order
0,0,APRIL,18,1
1,0,MAY,25,2
2,0,MARCH,18,3
3,0,SEPTEMBER,16,4
4,0,OCTOBER,20,5
5,0,AUGUST,7,6
6,0,JUNE,14,7
7,0,JANUARY,14,8
8,0,JULY,12,9
9,0,FEBRUARY,2,10


In [4]:
def generate_data(stop_proba_func, n=10000):
    """
    Options
    stop_proba_func: Takes in df of subject's 12 months and must return 12 probabilities of stopping *after* the current one
    """

    def random_stopped(df):
        stop_proba = stop_proba_func(df)
        df["last"] = [True if x == 1 else False for x in np.random.binomial([1]*12, stop_proba)]
        for i, x in enumerate(df["last"]):
            if x:
                df = df.iloc[:i+1]
                break
        return df

    return df.groupby("subject.id").apply(random_stopped).reset_index(drop=True)

In [5]:
raw_df.groupby("order").agg({"did_continue_eval": "mean"})

,did_continue_eval
order,
1.0,0.908309
2.0,0.859843
3.0,0.800366
4.0,0.694253
5.0,0.665563
6.0,0.570732
7.0,0.582609
8.0,0.552239
9.0,0.611111


In [6]:
# random based on real-life average stopping probabilities based on number evaluated so far

empirical_stop_proba = raw_df.groupby("order").agg({"last": "mean"})["last"].values

def stop_proba_random(df):
    return empirical_stop_proba

random = generate_data(stop_proba_random)
random.head(16)

random.to_csv("~/Desktop/thesis/data/generated_random.csv")

In [41]:
# optimal with no revisiting (assumption you can't go back)
# Explore until explore_index, then take next that is at least as good

def stop_proba_random(df):
    explore_index = 3 # 1 + the zero-indexed last index for explore before exploit
    highest = max(df["value"].iloc[:explore_index])
    stop_proba = [1 if i >= explore_index and x >= highest else 0 for i, x in enumerate(df["value"])]
    if sum(stop_proba) == 0:
        stop_proba[-1] = 1
    return stop_proba

optimal_no_revisit = generate_data(stop_proba_random)
optimal_no_revisit.head(16)

optimal_no_revisit.to_csv("~/Desktop/thesis/data/generated_optimal_no_revisit.csv")

# 169 have best value in the top 3 and thus have to go through all the months
#count

In [ ]:
# optimal as a function of most recent value?
# - need to assume some cost of time...

count = 0
def stop_proba_random(df):
    
    explore_index = 3 # 1 + the zero-indexed last index for explore before exploit
    highest = max(df["value"].iloc[:3])
    stop_proba = [1 if i >= explore_index and x >= highest else 0 for i, x in enumerate(df["value"])]
    if sum(stop_proba) == 0:
        global count
        count += 1
    return stop_proba

optimal_stopping = generate_data(stop_proba_random)
optimal_stopping.head(16)


In [ ]:
# optimal based on limited knowledge/set of variables?
# as a function of best value so far (interaction with order)


In [ ]:
order_filter = 2
filtered_result = result[result["order"]==order_filter]

minimum = min(month_values)
maximim = max(month_values)
non_last_eval = filtered_result.loc[filtered_result["did_continue_eval"], "highest_value_so_far"]
last_eval = filtered_result.loc[~filtered_result["did_continue_eval"], "highest_value_so_far"]

bins = np.linspace(minimum, maximim, 10)

plt.hist(non_last_eval, bins, alpha=0.5, label='continued evaluating', weights=np.ones(len(non_last_eval)) / len(non_last_eval))
plt.hist(last_eval, bins, alpha=0.5, label='stopped', weights=np.ones(len(last_eval)) / len(last_eval))
plt.legend(loc='upper left')
plt.xlabel("Value of best action so far")
plt.ylabel("Percentage of situations")
plt.show()